<a href="https://colab.research.google.com/github/quastarK/Tensorflow_colab/blob/Tensorflow-2.0-%EB%94%A5%EB%9F%AC%EB%8B%9D%EA%B8%B0%EC%B4%88/4_%EC%A6%89%EC%8B%9C%EC%8B%A4%ED%96%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

## 변수는 객체

즉시실행에서 변수는 그객체의 마지막 참조가 제거될때까지 유지되고 그 이후 삭제됨

In [2]:
if tf.config.experimental.list_physical_devices("GPU"):
  with tf.device("gpu:0"):
    print("GPU 사용 가능")
    v = tf.Variable(tf.random.normal([1000, 1000]))
    v = None  # v는 더이상 GPU 메모리를 사용하지 않음

GPU 사용 가능


In [6]:
print(v) #참조가 사라짐

None


## 객체 기반의 저장

모델의 상태를 저장할때 사용

tf.train.Checkpoint는 tf.Variable을 체크포인트 파일로 저장하거나 체크포인트 파일에서 복구할 수 있음

In [23]:
x = tf.Variable(10.0)
checkpoint = tf.train.Checkpoint(x=x)

print(x)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=10.0>


In [24]:
x.assign(2.)
checkpoint_path = './ckpt/'
checkpoint.save('./ckpt/')  # -1 이라는 이름으로 저장 된것
print(x)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>


In [25]:
x.assign(11.) # 저장한 후에 변수 변경
print(x)
# 체크포인트로부터 값을 복구 2로 저장되어 있었음
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path))

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=11.0>


In [19]:
print(x)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.0>


## 동적인 제어 흐름

모델을 저장하거나 읽어들이기 위해서 tf.train.Checkpoint는 숨겨진 변수를 요구하지 않고 객체 내부 상태를 저장합니다.

옵티마이저와 모델, 전역 단계 상태를 기록하려면 tf.train.Checkpoint에 전달하면 된다.

In [20]:
import os

model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(16,[3,3], activation='relu'),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(10)
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
checkpoint_dir = 'path/to/model_dir'
if not os.path.exists(checkpoint_dir):      # 폴더가 없으면 폴더 생성
  os.makedirs(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
root = tf.train.Checkpoint(optimizer=optimizer,     # Checkpoint에 model, optimizer 전달
                           model=model)

root.save(checkpoint_prefix)      # 저장 path/to/model_dir 안에 ckpt~ 이름으로 저장
root.restore(tf.train.latest_checkpoint(checkpoint_dir))

## 객체 지향형 지표

tf.keras.metrics는 객체로 저장됩니다.
새로운 데이터를 이 객체에 전달하여 지표를 수정하고 tf.keras.metrics.result 메서드를 사용해 그 결과를 얻음

In [27]:
m = tf.keras.metrics.Mean('loss')

In [28]:
m(0)    # 0 값이 들어감
m(5)  # 5 값이 들어감 -> [0,5]
m.result() # mean([0,5]) = 2.5

<tf.Tensor: shape=(), dtype=float32, numpy=2.5>

In [29]:
m([8, 9]) # [0, 5, 8, 9]
m.result()  #mean([0,5,8,9]) = 5.5

<tf.Tensor: shape=(), dtype=float32, numpy=5.5>

## 서머리 (summary)와 텐서 보드

텐서보드는 훈련과정에서 모델을 파악하거나 디버깅하고 최적화하기 위해 사용하는 시각화 도구입니다.
텐서보드는 프로그램이 실행되는 동안 작성된 서머리 이벤츠를 사용함


즉시 실행에서 변수의 서머리 정보를 기록하기 위해서 tf.summary를 사용함
예를 들어, 다음에 매 100번째 훈련마다 loss의 서머리 정보를 기록

In [32]:
logdir = "./tb/"
writer = tf.summary.create_file_writer(logdir)

with writer.as_default():
  for i in range(1000): # 또는 반복전에 writer.set_as_default()를 호출
    step = i+1
    # 실제 훈련 함수로 손실을 계산
    loss = 1- 0.001*step
    if step % 100 ==0:
      tf.summary.scalar('손실',loss, step=step)

## 사용자 정의 그래디언트

정방향 함숭란에서 입력값 or 출력값, 중간값과 관련된 그래디언트를 정의해야 함


In [33]:
@tf.custom_gradient
def clip_gradient_by_norm(x,norm):
  y = tf.identity(x)
  def grad_fn(dresult):
    return [tf.clip_by_norm(dresult,norm), None]  # clip_by_norm : gradient가 너무 커져 모델의 학습이 이상해지는 것 막음 (loss의 최대 최소값 설정, clip)
  return y, grad_fn

# 성능 체크




In [34]:
import time
time.time()

1706713602.4009411

In [35]:
def measure(x, steps):
  # 텐서플로는 처음 사용할 때 GPU를 초기화, 시간계산에서 제외
  tf.matmul(x, x)
  start = time.time()
  for i in range(steps):
    x = tf.matmul(x, x)
  # tf.matmul는 행렬 곱셈을 완료하기 전에 결과를 반환할 수 있습니다
  # (예, CUDA 스트림 대기열에 연산을 추가한 후에 결과를 반환할 수 있다).
  # 아래 x.numpy() 호출은 대기열에 추가된 모든 연산이 완료될 것임을 보장합니다
  # (그리고 그 결과가 호스트 메모리에 복사될 것이고,
  # 그래서 matnul 연산시간보다는 조금 많은 연산시간이
  # 포함됩니다).
  _ = x.numpy()
  end = time.time()
  return end - start


In [36]:
measure(tf.ones([1000,1000]), 1000)

0.5183589458465576

In [37]:
shape = (1000, 1000)
steps = 200
print("{} 크기 행렬을 자기 자신과 {}번 곱했을 때 걸리는 시간:".format(shape, steps))

# CPU에서 실행:
with tf.device("/cpu:0"):
  print("CPU: {} 초".format(measure(tf.random.normal(shape), steps)))

# GPU에서 실행, 가능하다면:
if tf.config.experimental.list_physical_devices("GPU"):
  with tf.device("/gpu:0"):
    print("GPU: {} 초".format(measure(tf.random.normal(shape), steps)))
else:
  print("GPU: 없음")

(1000, 1000) 크기 행렬을 자기 자신과 200번 곱했을 때 걸리는 시간:
CPU: 6.0387914180755615 초
GPU: 0.15822887420654297 초


In [ ]:
if tf.config.experimental.list_physical_devices("GPU"):
  x = tf.random.normal([1000, 1000])

  x_gpu0 = x.gpu()
  x_cpu = x.cpu()

  _ = tf.matmul(x_gpu0, x_gpu0)
  _ = tf.matmul(x_cpu, x_cpu)

In [ ]:
%timeit tf.matmul(x_gpu0, x_gpu0)

In [ ]:
%timeit tf.matmul(x_cpu, x_cpu)